<a href="https://colab.research.google.com/github/sayanbanerjee32/feedback-prize-effectiveness/blob/main/feedback_prize_v3_ulmfit_baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# install fastkaggle if not available
!pip install -Uq fastai
try: import fastkaggle
except ModuleNotFoundError:
    !pip install -Uq fastkaggle

# !pip install -Uq 'timm>=0.6.2.dev'
!pip install -Uq pynvml
from fastkaggle import *

     |████████████████████████████████| 46 kB 2.7 MB/s 


In [2]:
import os
from pathlib import Path
import fastai
fastai.__version__

'2.7.9'

In [7]:
# config depending on whether this is running on kaggle or collab
# is_colab = True
is_colab = not os.environ.get('KAGGLE_KERNEL_RUN_TYPE', '')
comp = 'feedback-prize-english-language-learning'
if is_colab:
    model_save_path = Path('/content/'+comp+'_out/models')
else:
    model_save_path = Path('/kaggle/working/'+comp) #+'/models')

In [4]:
# import colab libraries
if is_colab:
    from google.colab import drive
    drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# The Kaggle API client expects this file to be in ~/.kaggle,
# so lets move it there.
if is_colab:
    !mkdir ~/.kaggle
    !cp /content/drive/MyDrive/Kaggle_api_auth/kaggle.json ~/.kaggle/

In [6]:
# This permissions change avoids a warning on Kaggle tool startup.
if is_colab:
    !chmod 600 ~/.kaggle/kaggle.json

In [8]:
path = setup_comp(comp)
path

100%|██████████| 2.80M/2.80M [00:00<00:00, 188MB/s]

Path('feedback-prize-english-language-learning')

In [9]:
from fastai.text.all import *
set_seed(32)

### Language model for original text

In [11]:
# read train csv
train_df = pd.read_csv(path / 'train.csv')
train_df.head()

,text_id,full_text,cohesion,syntax,vocabulary,phraseology,grammar,conventions
0,0016926B079C,"I think that students would benefit from learning at home,because they wont have to change and get up early in the morning to shower and do there hair. taking only classes helps them because at there house they'll be pay more attention. they will be comfortable at home.\n\nThe hardest part of school is getting ready. you wake up go brush your teeth and go to your closet and look at your cloths. after you think you picked a outfit u go look in the mirror and youll either not like it or you look and see a stain. Then you'll have to change. with the online classes you can wear anything and st...",3.5,3.5,3.0,3.0,4.0,3.0
1,0022683E9EA5,When a problem is a change you have to let it do the best on you no matter what is happening it can change your mind. sometimes you need to wake up and look what is around you because problems are the best way to change what you want to change along time ago. A\n\nproblem is a change for you because it can make you see different and help you to understand how tings wok.\n\nFirst of all it can make you see different then the others. For example i remember that when i came to the United States i think that nothing was going to change me because i think that nothing was going to change me bec...,2.5,2.5,3.0,2.0,2.0,2.5
2,00299B378633,"Dear, Principal\n\nIf u change the school policy of having a grade b average that unfair. Because many students have a C average. So that means that they cant go out for sports or other activities they want to do bad. That's like taking everything they have. What if kids want to become good at something, but now they cant because of that school policy. If they have a C average they should still be able to go out for sports or activities. A C average isn't that bad, its higher then a D average. If the school police was if you have a D average of lower they shouldn't do sports or activities....",3.0,3.5,3.0,3.0,3.0,2.5
3,003885A45F42,"The best time in life is when you become yourself. I agree that the greatest accomplishment, is when you be yourself in a world that constantly trying to make you something else. Because you make your own choices, you become more happy, and you respect others.\n\nFirst, you make your own choices by being yourself. Becoming yourself means that you should be able to make your own choices and not be shy or afraid of what you're doing. Because you're defining yourself by doing those things that you want. Some people follow others, therefore, they don't make their own choices. People are afraid...",4.5,4.5,4.5,4.5,4.0,5.0
4,0049B1DF5CCC,Small act of kindness can impact in other people can change people to become better persons you can have an impact of kindess with a homeles that can change his life or with some who needed they are going to know you are a nice person if you are a nice person everywhere you go people is going to like your personality so you have to be a nice person with others like a old women triying to cross the road thats a impact of kindness when you do that you feel a greate person you can change people in the way they think by helping others treating nice other people give them some advice when you s...,2.5,3.0,3.0,3.0,2.5,2.5


In [10]:
# In this cell fastai TextDataLoader functions are overridden to allow taking output 
# directory as input so that the toknised data is not written in data input directory
# in Kaggle, data input directory is read-only
from fastai.text.core import _tokenize_files
@delegates(_tokenize_files)
def tokenize_folderOP(path, extensions=None, folders=None,
                      output_dir=None, skip_if_exists=True, **kwargs):
    """replacement of fastai.text.core.tokenize_folder so that it passes
     output_dir as input to _tokenize_files"""
    path,extensions = Path(path),ifnone(extensions, ['.txt'])
    files = get_files(path, extensions=extensions, recurse=True, folders=folders)
    def _f(i,output_dir): return output_dir/files[i].relative_to(path)
    return _tokenize_files(_f, files, path, output_dir, 
                           skip_if_exists=skip_if_exists, **kwargs)

class TokenizerOP(Tokenizer):
    """Wrapper class for fastai.text.core.Tokenizer class to override
    from_folder method to take output_dir as input and call the overridden
    method tokenize_folderOP"""
    def __init__(self, tok, rules=None, counter=None,
                 lengths=None, mode=None, sep=' '):
        super().__init__(tok, rules=rules, counter=counter,
                 lengths=lengths, mode=mode, sep=sep)
    
    @classmethod
    @delegates(tokenize_folderOP, keep=True)
    def from_folder(cls, path, tok=None, rules=None, output_dir = None, **kwargs):
        path = Path(path)
        if tok is None: tok = WordTokenizer()
        output_dir = tokenize_folderOP(path, tok=tok, rules=rules, 
                                       skip_if_exists = False, 
                                       output_dir = output_dir, **kwargs)
        res = cls(tok, counter=load_pickle(output_dir/fn_counter_pkl),
                  lengths=load_pickle(output_dir/fn_lengths_pkl),
                  rules=rules, mode='folder')
        res.path,res.output_dir = path,output_dir
        return res


class TextBlockOP(TextBlock):
    "Overriding TextBlock for user defined output dir"
    def __init__(self, tok_tfm, vocab=None, is_lm=False,
                 seq_len=72, backwards=False, **kwargs):
        super().__init__(tok_tfm, vocab=vocab, is_lm=is_lm,
                 seq_len=seq_len, backwards=backwards, **kwargs)
    @classmethod
    @delegates(TokenizerOP.from_folder, keep=True)
    def from_folder(cls, path, vocab=None, is_lm=False, seq_len=72,
                    backwards=False, min_freq=3, max_vocab=60000,output_dir =None,
                    **kwargs):
        "Build a `TextBlock` from a `path` - calls TokenizerOP and provides output_dir as input"
        return cls(TokenizerOP.from_folder(path, output_dir = output_dir, **kwargs), vocab=vocab, is_lm=is_lm, seq_len=seq_len,
                   backwards=backwards, min_freq=min_freq, max_vocab=max_vocab)
        
class TextDataLoadersOP(TextDataLoaders):
    "Basic wrapper around several `DataLoader`s with factory methods for NLP problems"
    @classmethod
    @delegates(DataLoaders.from_dblock)
    def from_folder(cls, path, train='train', valid='valid', valid_pct=None,
                    seed=None, vocab=None, text_vocab=None, is_lm=False,
                    tok_tfm=None, seq_len=72, splitter=None,
                    backwards=False, output_dir =None, **kwargs):
        "This is to override same method from TextDataLoaders to accept and pass output_dir as input"
        if splitter is None:
            splitter = GrandparentSplitter(train_name=train,
                                           valid_name=valid) if valid_pct is None else RandomSplitter(valid_pct,
                                                                                                      seed=seed)
        blocks = [TextBlockOP.from_folder(path, text_vocab, is_lm, seq_len, backwards,
                                          tok=tok_tfm, output_dir = output_dir)]
        if not is_lm: blocks.append(CategoryBlock(vocab=vocab))
        get_items = partial(get_text_files, folders=[train,valid]) if valid_pct is None else get_text_files
        dblock = DataBlock(blocks=blocks,
                           get_items=get_items,
                           splitter=splitter,
                           get_y=None if is_lm else parent_label)
        return cls.from_dblock(dblock, path, path=path, seq_len=seq_len, **kwargs)

    @classmethod
    @delegates(DataLoaders.from_dblock)
    def from_df(cls, df, path='.', valid_pct=0.2, seed=None, text_col=0, label_col=1, label_delim=None, y_block=None,
                text_vocab=None, is_lm=False, valid_col=None, tok_tfm=None, tok_text_col="text", seq_len=72, backwards=False, **kwargs):
        "Create from `df` in `path` with `valid_pct`"
        blocks = [TextBlock.from_df(text_col, text_vocab, is_lm, seq_len, backwards, tok=tok_tfm)]
        if y_block is None and not is_lm:
            blocks.append(MultiCategoryBlock if is_listy(label_col) and len(label_col) > 1 else CategoryBlock)
        if y_block is not None and not is_lm: blocks += (y_block if is_listy(y_block) else [y_block])
        splitter = RandomSplitter(valid_pct, seed=seed) if valid_col is None else ColSplitter(valid_col)
        dblock = DataBlock(blocks=blocks,
                           get_x=ColReader(tok_text_col),
                           get_y=None if is_lm else ColReader(label_col, label_delim=label_delim),
                           splitter=splitter)
        return cls.from_dblock(dblock, df, path=path, seq_len=seq_len, **kwargs)

In [17]:
tok_path = model_save_path / 'train_tok'
tok_path.mkdir(parents=True, exist_ok=True)
dls_lm = TextDataLoaders.from_df(train_df, text_col='full_text', is_lm=True,
                                       valid_pct=0.1, output_dir = tok_path)
dls_lm.show_batch(max_n=5)

,text,text_
0,"xxbos xxmaj the use of technology has a positive effects on people 's lives , it help us to get ready by doing our jobs faster , we do nt lose the much time in communicating , we do n't need to worry about losing paper work . \n\n xxmaj using technology makes our work easy and faster , it makes easy and fast by sending email , so that way some","xxmaj the use of technology has a positive effects on people 's lives , it help us to get ready by doing our jobs faster , we do nt lose the much time in communicating , we do n't need to worry about losing paper work . \n\n xxmaj using technology makes our work easy and faster , it makes easy and fast by sending email , so that way some important"
1,"just ca n't do it anymore , but students struggle with their own personal problems but they can get through it all and that is what makes the build - up self - confidence about the risk and task they make in life . xxbos xxmaj is enjoyable for everyone because you can do all sort of fun activities . xxmaj one way i like spending my time is playing soccer with","ca n't do it anymore , but students struggle with their own personal problems but they can get through it all and that is what makes the build - up self - confidence about the risk and task they make in life . xxbos xxmaj is enjoyable for everyone because you can do all sort of fun activities . xxmaj one way i like spending my time is playing soccer with all"
2,"to their career at a early age . \n\n xxmaj it is a great idea for students to commit to a career at a young age because student will know what type of courses are required for that particular career , students will not worried about what to do after graduation , and students will be able to develop some basic fundamentals skills needed for their future career . \n\n xxmaj initially","their career at a early age . \n\n xxmaj it is a great idea for students to commit to a career at a young age because student will know what type of courses are required for that particular career , students will not worried about what to do after graduation , and students will be able to develop some basic fundamentals skills needed for their future career . \n\n xxmaj initially ,"
3,"live with his parents no more . xxmaj this will help him to be responsible person because he has to take care of himself by his own . xxmaj another example can be , xxmaj in school , xxmaj students choose their classes for next year by themselves . xxmaj this will help them to develop their self confidence because they are deciding what is better for them or not by their","with his parents no more . xxmaj this will help him to be responsible person because he has to take care of himself by his own . xxmaj another example can be , xxmaj in school , xxmaj students choose their classes for next year by themselves . xxmaj this will help them to develop their self confidence because they are deciding what is better for them or not by their own"
4,"happy for him . xxmaj he did n't care if he is second or last . xxmaj he cares about his relative cheering for him . \n\n xxmaj conclusion , xxmaj failure plays in the pursuit of success . i believe failure helps people to find their a path to success . xxmaj people fail , but they can learn from their mistakes and it 's positively affects people personalty . xxmaj","for him . xxmaj he did n't care if he is second or last . xxmaj he cares about his relative cheering for him . \n\n xxmaj conclusion , xxmaj failure plays in the pursuit of success . i believe failure helps people to find their a path to success . xxmaj people fail , but they can learn from their mistakes and it 's positively affects people personalty . xxmaj such"


In [13]:
# dump vocab that will be required for inference script
with open(model_save_path / 'dls_lm_vocab.pickle', 'wb') as b:
    pickle.dump(dls_lm.vocab,b)

In [14]:
learn_lm = language_model_learner(dls_lm, AWD_LSTM, metrics=[accuracy, Perplexity()], path=path, wd=0.1).to_fp16()

In [15]:
learn_lm.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,3.906570,3.667725,0.277480,39.162693,01:19


In [ ]:
learn_lm.unfreeze()
learn_lm.fit_one_cycle(10, 1e-3)

In [16]:
model_save_path.mkdir(parents=True, exist_ok=True)
if iskaggle:
    # hack to save encoder in a writable location
    learn_lm.path = model_save_path
    learn_lm.save_encoder('finetuned_enc')
    # learn_lm.save('finetuned_lm')
else:
    learn_lm.save_encoder(model_save_path / 'finetuned_enc')